In [1]:
# Importing the StringIO module.
from bs4 import BeautifulSoup

# import the requests library 
import requests

import pandas as pd

import re

#Inserting stock data into SQL database using sqlachemy
from sqlalchemy import create_engine

#Defining headers with User agent to establish requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

import pandas as pd
import numpy as np

In [2]:

#lst = ['SBIN','TCS','HCLTECH','ROSSARI','PIDILITIND']

#Function to scrape Marketsmith stock fundamental data
def marketsmith_scrape(lst):
    for x in range(0,len(lst)):
        options = Options()
        options.add_argument('--headless')

        #Firefox browser webdriver
        driver = webdriver.Firefox(options=options)


        #Google chrome webdriver
        #options = webdriver.ChromeOptions()
        #driver = webdriver.Chrome(options=options)
        
        #Base Url to fetch stock fundamental data through request


        url = "https://marketsmithindia.com/mstool/eval/{}/evaluation.jsp#/".format(lst[x])
        print(url)
    
        driver.get(url)
        time.sleep(4)
        #soup = BeautifulSoup(driver.page_source, 'lxml')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        

        #Populating summary columns for individual stocks from Marketsmith evaluation pages
        if x==0:
            stock_detail_header = soup.find_all('div', class_ = "details")

        print(format(lst[x]),">>>" ,stock_detail_header)

        stock_detail_value = soup.find_all('div', class_ = "value")
        print(format(lst[x]),">>>",stock_detail_value)
        
        cols = []
        
        
        #Generating columns through iteration
        
        for y in range(0,len(stock_detail_header)):
            cols.append(stock_detail_header[y].get_text().strip())
            
        #Creating a new column for stock wise segregation
        cols.insert(0,"Stock Name")
        
        print(y,"columns","--->>>",cols)
        
        
        #share_summary = pd.DataFrame(columns= cols)
        
        
        #Feeding an empty Dataframe with its respective column values through iterating summary_data list
        #Creating an empty Dataframe to feed the column names through iterating summary_cols list

        summary_data_values = []
        

        #Generating columns data through iteration
        for z in range(0,len(stock_detail_value)):
            print(stock_detail_value[z].get_text().strip())
            summary_data_values.append(stock_detail_value[z].get_text().strip())
        
        #Adding stock name as a new column to every data rows for identification.
        summary_data_values.insert(0,lst[x])       
        
        print(z,"summary_data_values","--->>>",summary_data_values)    
        
        #Creating base data frame to hold data for every stock
        share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
        
        #Removing duplicate rows to avoid discrepency
        share_summary = share_summary.loc[:,~share_summary.columns.duplicated()]
        
        #Creating final Data frame one time and from base dataframe structure
        if x==0:
            #share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
            
            all_share_summary = pd.DataFrame(columns= [share_summary])
        print ("Inside x loop",x)
            
        #Adding individual stock data values through every iteration    
        print("Final Datatypes","<<<<<<<<->>>>>>>>",all_share_summary.dtypes)
        all_share_summary = all_share_summary._append(share_summary,ignore_index=True)
        
        #del(stock_detail_header,stock_detail_value,share_summary)
        summary_data_values.clear()


        #Deleting objects cache to contents to store data in next iteration
        soup.decompose()
        
        #Closing the driver    
        driver.quit()
        
        print("Next Iteration")
    
    

    #Data cleaning - Dropping unncessary columns
    all_share_summary.drop(list(all_share_summary)[0:23], axis=1,inplace = True)
    
    #Renaming column name for SQL table processing
    all_share_summary.rename(columns = {'Stock Name' : 'stock_name', 'Market Capitalization' : 'market_capitalization', 'Sales' : 'sales', \
        'Shares in Float' : 'shares_in_float', 'No of Funds' : 'no_of_funds', 'Shares held by Funds' : 'shares_held_by_funds', 'Yield' : 'yield', \
        'Book Value' : 'book_value', 'U/D Vol Ratio' : 'u/d_vol_ratio', 'LTDebt/Equity' : 'ltdebt_equity', 'Alpha' : 'alpha', 'Beta' : 'beta', \
        'Master Score' : 'master_score', 'EPS Rating' : 'eps_rating', 'Price Strength' : 'price_strength', 'Acc/Dis Rating' : 'buyers_demand', \
        'Group Rank' : 'group_rank_out_of_197', 'EPS Growth Rate' : 'eps_growth_rate', 'Earnings Stability' : 'earnings_stability', 'P/E Ratio' : 'pe_ratio', \
        '5-Year P/E Range' : '5years_pe_range', 'Return on Equity' : 'return_on_equity', 'Cash Flow (INR)': 'cash_flow'},
        inplace = True)
    
    #Removing INR string (text cleansing) from few columns to fit into perfect standard for data manipulation
    all_share_summary.market_capitalization = all_share_summary.market_capitalization.str.replace('INR ','')
    all_share_summary.sales = all_share_summary.sales.str.replace('INR ','')
    all_share_summary.group_rank_out_of_197 = all_share_summary.group_rank_out_of_197.str.replace(' of 197','')
    
    
    #Changing datatypes for the columns to their appropraite type for further processing
    all_share_summary.master_score = all_share_summary.master_score.astype('Int32')
    all_share_summary.eps_rating = all_share_summary.eps_rating.astype('Int32')
    all_share_summary.price_strength = all_share_summary.price_strength.astype('Int32')
    #all_share_summary.earnings_stability = all_share_summary.earnings_stability.astype('Int32')
    all_share_summary.pe_ratio = all_share_summary.pe_ratio.astype('Int32')
    all_share_summary.group_rank_out_of_197 = all_share_summary.group_rank_out_of_197.astype('Int32')

    #Inserting stock data into SQL database using sqlachemy
    
    #Defining table name on MSSQL server to locate the data
    table_name = "market_smith_stock_eval"
    
    sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')

    db_conn= sql_engine.connect()

    #DB Actions to load data from Pandas Dataframe to MSSQL
    try:
        all_share_summary.to_sql(table_name, db_conn, if_exists= 'replace',index= False)
    except Exception as ex:
        print(ex)
    else:
        print('Marketsmith Stock Evalulation data successfully inserted into MS SQL table-{}'.format(table_name))
    finally:
        db_conn.close()
    
    return all_share_summary


In [3]:
#lst = ['M&M']

lst =['ADANIENT','ADANIPORTS','APOLLOHOSP','ASIANPAINT','AXISBANK','BAJAJ-AUTO','BAJAJFINSV','BAJFINANCE','BHARTIARTL','BPCL','BRITANNIA','CIPLA','COALINDIA',
'DIVISLAB','DRREDDY','EICHERMOT','GRASIM','HCLTECH','HDFCBANK','HDFCLIFE','HEROMOTOCO','HINDALCO','HINDUNILVR','ICICIBANK','INDUSINDBK','INFY','ITC','JSWSTEEL',
'KOTAKBANK','LT','LTIM','M&M','MARUTI','NESTLEIND','NTPC','ONGC','POWERGRID','RELIANCE','SBILIFE','SBIN','SHRIRAMFIN','SUNPHARMA','TATACONSUM',
'TATAMOTORS','TATASTEEL','TCS','TECHM','TITAN','ULTRACEMCO','WIPRO',]

#Calling webscraping function
marketsmith_scrape(lst)

https://marketsmithindia.com/mstool/eval/ADANIENT/evaluation.jsp#/
ADANIENT >>> [<div class="details"> Market Capitalization</div>, <div class="details"> Sales </div>, <div class="details"> Shares in Float </div>, <div class="details"> No of Funds </div>, <div class="details"> Shares held by Funds </div>, <div class="details"> Market Capitalization </div>, <div class="details"> Sales </div>, <div class="details"> Shares in Float </div>, <div class="details"> No of Funds </div>, <div class="details"> Shares held by Funds </div>, <div class="details"> Yield </div>, <div class="details"> Book Value </div>, <div class="details"> U/D Vol Ratio </div>, <div class="details"> LTDebt/Equity </div>, <div class="details"> Alpha </div>, <div class="details"> Beta </div>, <div class="details"> Master Score </div>, <div class="details"> EPS Rating </div>, <div class="details"> Price Strength </div>, <div class="details"> Acc/Dis Rating </div>, <div class="details"> Group Rank </div>, <div class="det

,stock_name,market_capitalization,sales,shares_in_float,no_of_funds,shares_held_by_funds,yield,book_value,u/d_vol_ratio,ltdebt_equity,...,eps_rating,price_strength,buyers_demand,group_rank_out_of_197,eps_growth_rate,earnings_stability,pe_ratio,5years_pe_range,return_on_equity,cash_flow
0,ADANIENT,"3,45,306.01 Cr",44580.47 Cr,30.78 Cr,533 (+13.89 %),23.00 Cr (+0.41 %),0.04%,24.99x,1.1,6%,...,79,54,B-,114,14%,99,139,16-\n 397,12%,98.13
1,ADANIPORTS,"2,89,782.52 Cr",6226.89 Cr,73.44 Cr,736 (+13.58 %),57.90 Cr (+0.31 %),0.38%,10.03x,1.2,154%,...,78,77,B,31,N/A,43,219,31-\n 569,-2%,12.56
2,APOLLOHOSP,"85,566.45 Cr",7020.03 Cr,10.21 Cr,1.38 K (+6.47 %),9.37 Cr (+0.08 %),0.24%,12.35x,0.6,22%,...,90,42,C-,68,15%,53,86,53-\n 446,16%,85.16
3,ASIANPAINT,"2,84,584.31 Cr",30995.20 Cr,45.08 Cr,1.07 K (-5.82 %),26.43 Cr (-1.25 %),0.93%,17.7x,0.9,0%,...,90,17,D-,163,6%,21,52,61-\n 110,26%,44.01
4,AXISBANK,"3,55,033.97 Cr",131810.59 Cr,284.03 Cr,1.67 K (+0.18 %),250.14 Cr (+0.36 %),0.09%,2.4x,1.1,131%,...,91,50,B,85,33%,60,14,14-\n 123,17%,87.43
5,BAJAJ-AUTO,"2,53,662.96 Cr",44685.23 Cr,12.56 Cr,1.08 K (+8.76 %),6.44 Cr (-2.74 %),0.89%,10x,1.5,N.A,...,80,79,A-,49,2%,26,34,17-\n 35,30%,194.80
6,BAJAJFINSV,"2,57,676.86 Cr",1699.29 Cr,62.12 Cr,713 (+3.03 %),25.13 Cr (+2.26 %),0.05%,48.44x,1.3,N.A,...,87,32,C,97,8%,58,222,145-\n 1128,14%,4.95
7,BAJFINANCE,"4,26,116.51 Cr",46932.68 Cr,27.85 Cr,1.20 K (-3.99 %),21.64 Cr (+0.04 %),0.53%,7.9x,1.2,221%,...,89,24,B,109,20%,33,33,37-\n 110,20%,-451.43
8,BHARTIARTL,"7,39,457.84 Cr",91603.40 Cr,271.27 Cr,1.53 K (+10.69 %),264.27 Cr (+3.75 %),0.3%,9.54x,1.5,173%,...,34,68,A+,52,4%,39,167,129-\n 936,0%,78.17
9,BPCL,"1,37,823.43 Cr",449570.11 Cr,101.95 Cr,1.27 K (+10.96 %),82.63 Cr (+4.82 %),4.1%,2.49x,1.1,37%,...,63,75,B,26,N/A,82,4,3-\n 85,4%,50.08
